# Project Introduction: Integrating Moving Average (MA) Crossover and Average Directional Index (ADX) for Stock Analysis

## Objective
The objective of this project is to develop a reliable stock screening tool that identifies potential buy and sell signals using two powerful technical indicators: the Moving Average (MA) Crossover and the Average Directional Index (ADX). By combining these indicators, we aim to enhance the accuracy of our signals, leveraging the strengths of both trend identification and trend strength analysis.

## Background
Technical analysis involves using historical price and volume data to forecast future price movements. Among the myriad of technical indicators available, the MA Crossover and ADX are two widely respected tools that traders use to assess market conditions.

## Why Moving Average Crossover and ADX?

### Moving Average (MA) Crossover
- **Purpose:** The MA Crossover is used to identify potential trend reversals by examining the crossing points of different moving averages.
- **Components:**
  - **Short-Term MA:** A moving average calculated over a shorter period (e.g., 20-day MA).
  - **Long-Term MA:** A moving average calculated over a longer period (e.g., 100-day MA).
- **Signals:**
  - **Potential Bullish Trend Reversal:** Occurs when the short-term MA crosses above the long-term MA, indicating a potential buy signal.
  - **Potential Bearish Trend Reversal:** Occurs when the short-term MA crosses below the long-term MA, indicating a potential sell signal.

### Average Directional Index (ADX)
- **Purpose:** The ADX is used to quantify the strength of a trend, regardless of its direction. It helps traders understand whether a market is trending or ranging.
- **Components:**
  - **ADX Line:** Indicates the strength of a trend. A value above 25 typically signifies a strong trend, while a value below 20 indicates a weak or non-trending market.
  - **Plus Directional Indicator (+DI):** Measures the presence of upward trend movement.
  - **Minus Directional Indicator (-DI):** Measures the presence of downward trend movement.
- **Signals:**
  - **Strong Trend:** ADX above 25.
  - **Weak Trend:** ADX below 20.

## Combining Moving Average Crossover and ADX
Using these indicators together can help confirm signals and reduce false positives:
- **Enhanced Accuracy:** While the MA Crossover helps identify trend reversals, ADX provides insight into the strength of the trend, filtering out weak signals in low trend strength environments.
- **Filter for False Signals:** The MA Crossover can sometimes give premature signals in weak trends. By requiring confirmation through ADX's trend strength measure, we filter out some of these false signals, increasing the likelihood that our signals are accurate.

## Methodology

- **Data Collection:** We collect historical price data for a selection of stocks over a specified period.
- **Indicator Calculation:** We calculate the MA Crossover and ADX for each stock.
- **Stock Screening:** We screen stocks based on the following criteria:
  - **Buy Criteria:** Short-term MA crosses above the long-term MA (Potential Bullish Trend Reversal), with ADX above 20 indicating a strong trend.
  - **Sell Criteria:** Short-term MA crosses below the long-term MA (Potential Bearish Trend Reversal), with ADX above 20 indicating a strong trend.
- **Visualization and Export:** We visualize the results and export the screened stocks to a CSV file for further analysis.

## Data Source and Flexibility
In this analysis, we will be using data from the FTSE 250 companies. However, feel free to use any dataset you prefer by simply changing the CSV file name in the script. Ensure that your dataset contains a column named "Ticker".

## Conclusion
This project aims to leverage the complementary strengths of the MA Crossover and ADX to create an intriguing stock screening tool. By considering both trend identification and trend strength, our signals offer interesting insights designed to inspire further analysis. The chosen thresholds and periods are standard and widely accepted in technical analysis, providing a thought-provoking starting point for potential future exploration.


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import contextlib
import sys
import os

# Function to suppress print statements
@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

# Function to calculate Moving Averages and detect crossovers
def calculate_moving_averages(data, short_window=10, long_window=50):
    data['Short_MA'] = data['Close'].rolling(window=short_window).mean()
    data['Long_MA'] = data['Close'].rolling(window=long_window).mean()
    data['Signal'] = np.where(data['Short_MA'] > data['Long_MA'], 1, 0)
    data['Crossover'] = data['Signal'].diff()
    return data

# Function to calculate ADX (Average Directional Index)
def calculate_adx(data, window=14):
    data['High_Low'] = data['High'] - data['Low']
    data['High_PrevClose'] = np.abs(data['High'] - data['Close'].shift())
    data['Low_PrevClose'] = np.abs(data['Low'] - data['Close'].shift())
    data['True_Range'] = data[['High_Low', 'High_PrevClose', 'Low_PrevClose']].max(axis=1)
    
    data['UpMove'] = data['High'] - data['High'].shift()
    data['DownMove'] = data['Low'].shift() - data['Low']
    data['Plus_DM'] = np.where((data['UpMove'] > data['DownMove']) & (data['UpMove'] > 0), data['UpMove'], 0)
    data['Minus_DM'] = np.where((data['DownMove'] > data['UpMove']) & (data['DownMove'] > 0), data['DownMove'], 0)
    
    data['ATR'] = data['True_Range'].rolling(window=window).mean()
    data['Smooth_Plus_DM'] = data['Plus_DM'].rolling(window=window).mean()
    data['Smooth_Minus_DM'] = data['Minus_DM'].rolling(window=window).mean()
    
    data['Plus_DI'] = (data['Smooth_Plus_DM'] / data['ATR']) * 100
    data['Minus_DI'] = (data['Smooth_Minus_DM'] / data['ATR']) * 100
    
    data['DX'] = np.abs(data['Plus_DI'] - data['Minus_DI']) / (data['Plus_DI'] + data['Minus_DI']) * 100
    
    data['ADX'] = data['DX'].rolling(window=window).mean()
    
    data.drop(['High_Low', 'High_PrevClose', 'Low_PrevClose', 'True_Range', 'UpMove', 'DownMove',
               'Plus_DM', 'Minus_DM', 'ATR', 'Smooth_Plus_DM', 'Smooth_Minus_DM', 'Plus_DI', 'Minus_DI', 'DX'], axis=1, inplace=True)
    
    return data

# Function to analyze stock for crossovers and ADX trend strength
def analyze_stock_ma_adx(symbol):
    try:
        end_date = datetime.now() - timedelta(days=1)
        start_date = end_date - timedelta(days=9*30)  
        
        with suppress_output():
            data = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
        
        if data.empty:
            print(f"No data fetched for {symbol}")
            return {"Symbol": symbol, "Data": pd.DataFrame()}

        data = calculate_moving_averages(data)
        data = calculate_adx(data)
        data.dropna(subset=['Short_MA', 'Long_MA', 'Crossover', 'ADX'], inplace=True)

        if data.empty:
            print(f"Insufficient data after calculating indicators for {symbol}")
            return {"Symbol": symbol, "Data": pd.DataFrame()}

        recent_crossover = data['Crossover'].iloc[-5:].values
        recent_adx = data['ADX'].iloc[-5:]
        bullish_trend_change = 1 in recent_crossover
        bearish_trend_change = -1 in recent_crossover
        strong_trend = data['ADX'].iloc[-1] > 25

        meets_criteria = (bullish_trend_change or bearish_trend_change) and strong_trend

        return {
            "Symbol": symbol,
            "Short_MA": data['Short_MA'].iloc[-1],
            "Long_MA": data['Long_MA'].iloc[-1],
            "ADX": data['ADX'].iloc[-1],
            "Bullish Trend Change": bullish_trend_change,
            "Bearish Trend Change": bearish_trend_change,
            "Strong Trend": strong_trend,
            "Meets Criteria": meets_criteria,
            "Data": data
        }

    except Exception as e:
        print(f"Error analyzing stock {symbol}: {e}")
        return {"Symbol": symbol, "Data": pd.DataFrame()}

# Load tickers from the CSV file
tickers_df = pd.read_csv('ftse_250_tickers.csv')
symbols = tickers_df['Ticker'].tolist()

# Initialize list to store results
results = []

# Analyze each stock
for symbol in tqdm(symbols, desc="Processing stocks"):
    stock_data = analyze_stock_ma_adx(symbol)
    if stock_data is not None and stock_data["Meets Criteria"]:
        results.append(stock_data)

# Convert results to DataFrame
results_df = pd.DataFrame([{
    "Symbol": r["Symbol"],
    "Short_MA": r["Short_MA"],
    "Long_MA": r["Long_MA"],
    "ADX": r["ADX"],
    "Bullish Trend Change": r["Bullish Trend Change"],
    "Bearish Trend Change": r["Bearish Trend Change"],
    "Strong Trend": r["Strong Trend"],
    "Meets Criteria": r["Meets Criteria"]
} for r in results])

# Export results to CSV
results_df.to_csv("ma_cross_adx_filtered_results.csv", index=False)

# Print results
results_df


In [ ]:
# Calculate and print the detailed results
total_tickers = len(tickers_df)
tickers_meeting_criteria = len(results_df)
percentage_meeting_criteria = (tickers_meeting_criteria / total_tickers) * 100

# Print results
print("My preference is a 5-20% threshold which strikes a balance, ensuring a\nfocused yet diverse selection of stocks to allow further investigations.")
print(f"Tickers meeting criteria: {tickers_meeting_criteria}")
print(f"Percentage of shares that pass the criteria: {percentage_meeting_criteria:.2f}%")

# Determine if the percentage is within the desired filter range
if 5 <= percentage_meeting_criteria <= 20:
    print("The percentage of shares meeting the criteria is within the desirable range (5-20%).")
else:
    print("The percentage of shares meeting the criteria is outside the desirable range (5-15%).")

In [ ]:
# Function to print basic financial data
def print_financial_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    def format_value(value, currency=False):
        if isinstance(value, (int, float)):
            formatted_value = f"{value:,}"
            if currency:
                formatted_value = f"£{formatted_value}"
            return formatted_value
        return value

    market_cap = format_value(info.get('marketCap', 'N/A'), currency=True)
    shares_outstanding = format_value(info.get('sharesOutstanding', 'N/A'))
    year_high = info.get('fiftyTwoWeekHigh', 'N/A')
    year_low = info.get('fiftyTwoWeekLow', 'N/A')
    pe_ratio = info.get('trailingPE', 'N/A')
    eps = info.get('trailingEps', 'N/A')
    dividend_yield = info.get('dividendYield', 'N/A')
    payout_ratio = info.get('payoutRatio', 'N/A')
    revenue = format_value(info.get('totalRevenue', 'N/A'), currency=True)
    gross_profit = format_value(info.get('grossProfits', 'N/A'), currency=True)
    net_income = format_value(info.get('netIncomeToCommon', 'N/A'), currency=True)
    total_debt = format_value(info.get('totalDebt', 'N/A'), currency=True)
    operating_cash_flow = format_value(info.get('operatingCashflow', 'N/A'), currency=True)
    free_cash_flow = format_value(info.get('freeCashflow', 'N/A'), currency=True)
    sector = info.get('sector', 'N/A')
    industry = info.get('industry', 'N/A')
    description = info.get('longBusinessSummary', 'N/A')

    print(f"{'Ticker:':<25} {ticker}")
    print(f"{'Company:':<25} {info.get('shortName', 'N/A')}")
    print(f"{'Sector:':<25} {sector}")
    print(f"{'Industry:':<25} {industry}")
    print(f"{'Market Cap:':<25} {market_cap}")
    print(f"{'Shares Outstanding:':<25} {shares_outstanding}")
    print(f"{'52-Week High:':<25} {year_high}")
    print(f"{'52-Week Low:':<25} {year_low}")
    print(f"{'PE Ratio (TTM):':<25} {pe_ratio}")
    print(f"{'EPS (TTM):':<25} {eps}")
    print(f"{'Dividend Yield:':<25} {dividend_yield}")
    print(f"{'Payout Ratio:':<25} {payout_ratio}")
    print(f"{'Revenue (TTM):':<25} {revenue}")
    print(f"{'Gross Profit (TTM):':<25} {gross_profit}")
    print(f"{'Net Income (TTM):':<25} {net_income}")
    print(f"{'Total Debt:':<25} {total_debt}")
    print(f"{'Operating Cash Flow (TTM):':<25} {operating_cash_flow}")
    print(f"{'Free Cash Flow (TTM):':<25} {free_cash_flow}")
    print(f"\n{'Description:':<25} {description}")  # Print the full description
    print("-" * 50)

# Function to plot MA and ADX indicators
def plot_ma_adx(data, symbol):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

    # Plot Close Price and Moving Averages
    ax1.plot(data.index, data['Close'], label='Close Price', color='blue')
    ax1.plot(data.index, data['Short_MA'], label='Short MA', color='red')
    ax1.plot(data.index, data['Long_MA'], label='Long MA', color='green')
    ax1.set_title(f'{symbol} - Close Price and Moving Averages')
    ax1.set_ylabel('Close Price')
    ax1.legend(loc='upper left')
    ax1.grid(True)

    # Plot ADX
    ax2.plot(data.index, data['ADX'], label='ADX', color='black')
    ax2.axhline(y=25, color='r', linestyle='--', label='ADX Threshold')
    ax2.set_title('ADX')
    ax2.set_ylabel('ADX Value')
    ax2.legend(loc='upper left')
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

print("The following stocks have met the specified criteria. A plot is displayed showing the stock price and its relationship to the associated indicators. Additionally, detailed financial and company information is provided for each stock.\n")


# Iterate through all tickers to plot graphs and print financial data
for stock_data in results:
    symbol = stock_data["Symbol"]
    data = stock_data["Data"]
    plot_ma_adx(data, symbol)
    print_financial_data(symbol)
